In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import xarray as xr
import load_in_beam
import FSDutil

# Use seasborn for nicer looking inline plots if available 
#import seaborn as sns
#sns.set(context='notebook', style='darkgrid')
#st = axes_style("whitegrid")


In [2]:
ATL07file='processed_ATL07-01_20181121134031_08240101_001_01.h5'
# ATL07file='processed_ATL07-01_20181206115828_10520101_001_01.h5'
localFilePath='/home/jovyan/Data/'+ATL07file

print(localFilePath)

beam_xarr = load_in_beam.read_beam(filename=localFilePath,beam='gt1r')

/home/jovyan/Data/processed_ATL07-01_20181121134031_08240101_001_01.h5
/home/jovyan/Data/processed_ATL07-01_20181121134031_08240101_001_01.h5


In [73]:
# This code is essentially FSDutil.get_CLD

isita_floe = 1 - beam_xarr.isita_lead
# print(len(q))

isitgood = beam_xarr.quality_flag

isitgood = np.concatenate([[0],isitgood])
startvec = np.concatenate([[0],isita_floe])
endvec = np.concatenate([isita_floe, [0]])

up = (startvec[:-1]==0) & (startvec[1:]==1)
down = (endvec[:-1]==1) & (endvec[1:]==0)        

baddy = []
for i in range(len(up)-1):
    if (down[i]) & (up[i+1]):
        baddy.append(i) 
        
print(baddy)        
        
startind = np.concatenate(np.where(up==1))
endind = np.concatenate(np.where(down==1))

print('original lengths: ' + str(len(startind)))

# Eliminate all isolated single ice measurements
# These are all where up and down are on the same point
same = np.intersect1d(startind,endind)

# These are all those where there is one single measurement
# but the quality of that measurement is high
good_same = np.where(isitgood==1)

same = np.setdiff1d(same,good_same)

startind = np.setdiff1d(startind,same)
endind = np.setdiff1d(endind,same)
startind = startind - 1 # because we prepended a zero

print('Num after removing bad and isolated "floes": ' + str(len(startind)))

startx = beam_xarr.seg_dist[startind]

startlat = beam_xarr.lat[startind]
startlat = startlat.drop(('lon','time','delta_time','lon360','segs'))

endlat = beam_xarr.lat[endind]
endlat = endlat.drop(('lon','time','delta_time','lon360','segs'))
chordlat = 0.5*(startlat+endlat)

startlon = beam_xarr.lon[startind]
startlon = startlon.drop(('lat','time','delta_time','lon360','segs'))

endlon = beam_xarr.lon[endind]
endlon = endlon.drop(('lat','time','delta_time','lon360','segs'))
chordlon = 0.5*(startlon+endlon)

startx = startx.drop(('lat','lon','time','delta_time','lon360','segs'))
endx = beam_xarr.seg_dist[endind]
endx = endx.drop(('lat','lon','time','delta_time','lon360','segs'))

chord_lengths = endx - startx
chord_lengths = chord_lengths[chord_lengths > 0]/1000

# plt.plot(chord_lengths)

[]
original lengths: 346
Num after removing bad and isolated "floes": 346
